In [9]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [13]:
##Paths
train_aud = '/Users/tacha/iu_research/speech_recognition/asr_workshop/data/target-segments/ru/clips/'
train_df = '/Users/tacha/iu_research/speech_recognition/asr_workshop/data/target-segments/ru/train.tsv'

**Transform functions**

Transform functions are the functions that are dealing with necessary input transformations e.g. feature extraction. They are feeded directly in the data loader. It helps to speed up data manipulation in contrast to reading all the file from the hard drive.

In [129]:
def find_maxlen(path, train_df):
    fnames = pd.read_csv(train_df, sep='\t')['path']
    maxlen = 0
    for n in tqdm(fnames):
        waveform, sample_rate = torchaudio.load(os.path.join(path, n))
        mfcc = torchaudio.transforms.MFCC()(waveform)
        size = mfcc.shape[2]
        if size > maxlen:
            maxlen = size
    print("Maxlen:", maxlen)


def extract_feats(path, maxlen=1083):
    '''
    Reads and processes one file at a time.
    Args:
        path: path to the file
        maxlen: maximum length of the spectrogram for padding
    '''
    waveform, sample_rate = torchaudio.load(path)
    #Calculate MFCC
    mfcc = torchaudio.transforms.MFCC()(waveform)
    #Calculate delta and double-delta
    deltas = torchaudio.transforms.ComputeDeltas()(mfcc)
    ddeltas = torchaudio.transforms.ComputeDeltas()(deltas)
    res = torch.cat((mfcc, deltas, ddeltas), dim=1).squeeze(0)
    #Normalize rows
    s = torch.sum(res, dim=1, keepdim=True)
    norm = torch.div(res, s)
    mask = torch.ones(norm.shape[0], norm.shape[1])
    padded_norm = nn.functional.pad(norm, pad=(0, maxlen-norm.shape[1], 0, 0), 
                                          mode="constant",value=0)
    padded_mask = nn.functional.pad(mask, pad=(0, maxlen-mask.shape[1], 0, 0), 
                                          mode="constant",value=0)
    return padded_norm, padded_mask

In [130]:
class TrainData(data.Dataset):
    def __init__(self, csv_path, aud_path, transform):
        self.df = pd.read_csv(csv_path, sep='\t')
        self.aud_path = aud_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        fname = os.path.join(self.aud_path, self.df['path'][idx])
        transcript = self.df['sentence'][idx]

        feat, mask = self.transform(fname)

        sample = {'aud':feat, 'trans': transcript, 'mask':mask}
        return sample

In [131]:
cv_dataset = TrainData(train_df, train_aud, extract_feats)

In [132]:
for i in cv_dataset:
    print(i['aud'].shape)

/Users/tacha/miniconda3/envs/prog/lib/python3.8/site-packages/torchaudio/functional.py:317: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120,

torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120, 1083])
torch.Size([120,

KeyError: 518